In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.SupportResistanceStrategy import SMAScanner
from ta.volatility import average_true_range
from src.classify import identify_pivots, calculate_scaled_slope

In [2]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

df_raw = yf.download(watchlist.split(' '), auto_adjust=False, interval='1d', period='6mo')

[*********************100%***********************]  80 of 80 completed


In [3]:
df = df_raw.stack([0,1], future_stack=True).reset_index().copy()
df.columns = ['Timestamp', 'Variable', 'Ticker', 'Value']
df = df.pivot(index=['Timestamp', 'Ticker'], columns='Variable', values='Value').reset_index().sort_values('Timestamp')
df.columns.name = None
# price = df.groupby('Ticker')['Close']
df_tick = df[df['Ticker'] == 'SPY'].copy()

In [4]:
tick_list = ['SPY', 'PG', 'META', 'AVGO', 'WMT', 'JPM', 'GLD', 'AAPL']
for tick in tick_list:
    df_tick = df[df['Ticker'] == tick].copy()
    scaled_slope = calculate_scaled_slope(df_tick, lookback=15)
    sentiment = None
    if scaled_slope >= 0.2: sentiment = 'bullish'
    elif (scaled_slope < 0.2) and (scaled_slope >= 0.1): sentiment = 'non-bearish'
    elif (scaled_slope < 0.1) and (scaled_slope >= -0.1): sentiment = 'neutral'
    elif (scaled_slope < 0) and (scaled_slope >= -0.2): sentiment = 'non-bullish'
    elif scaled_slope < -0.2: sentiment = 'bearish' 

    print(tick, scaled_slope, sentiment)

SPY 0.22338220849560092 bullish
PG -0.10131398961762011 non-bullish
META 0.13915189351985868 non-bearish
AVGO 0.17104480466177538 non-bearish
WMT 0.022277506073828862 neutral
JPM 0.2794374229675712 bullish
GLD -0.05448807262263508 neutral
AAPL 0.10537759651356446 non-bearish


In [ ]:
df_tick = df[df['Ticker'] == 'META'].copy()
lookback = 10 # periods
atr = average_true_range(df_tick['High'], df_tick['Low'], df_tick['Close'], window=lookback).to_numpy()

close = df_tick.sort_values('Timestamp')['Close']
sma = close.rolling(lookback).mean().iloc[-lookback:].to_numpy()
raw_slope = np.polyfit(sma[-lookback:], range(lookback), deg=1)[0]
raw_slope/atr[-1]

In [ ]:
df = identify_pivots(df_tick, threshold='atr',use_high_low=True)

In [ ]:
df.dropna()

# Later

In [ ]:
with open('cfg.txt') as f:

    token = f.read()

In [ ]:
import requests

url = "https://api.orats.io/datav2/hist/dailies"

querystring = {"token":token,"ticker":"AAPL","tradeDate":"2019-11-29"}

response = requests.request("GET", url, params=querystring)

print(response.text)


In [ ]:
import json
with open('backtest.json', 'r') as file:
    backtest = json.load(file)

headers = {
    "Authorization": token
}

response = requests.post(
    "https://api.orats.io/backtest/submit",
    headers=headers,
    data=backtest
)

In [ ]:
print("Status Code:", response.status_code)
print("Response Body:", response.text)

In [ ]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

df_raw = yf.download(watchlist.split(' '), auto_adjust=False, period='2y')

In [ ]:
scanner = SMAScanner(sma_periods=[50, 200], return_window=20)
output = scanner.run_scan(df_raw) # type: ignore
output

In [ ]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

df_raw = yf.download(watchlist.split(' '), auto_adjust=False, interval='60m', period='3mo')

In [ ]:
df = df_raw.stack(level=[0,1], future_stack=True).reset_index()
df.columns=['Timestamp', 'Variable', 'Ticker', 'Value']
df

In [ ]:
# assert set(df.columns) >= {'Timestamp', 'Variable', 'Ticker', 'Value'}
df_wide = df.pivot(index=['Timestamp', 'Ticker'], columns='Variable', values='Value').reset_index()
results = {}

window = 33
buffer = 0.01
for ticker, df_ticker in df_wide.groupby('Ticker'):
    df_ticker = df_ticker.sort_values('Timestamp').copy()
    df_ticker.set_index("Timestamp", inplace=True)
    if 'Open' not in df_ticker or 'Close' not in df_ticker:
        continue
    df_ticker = df_ticker.dropna(subset=['Open', 'Close'])
    body_top = df_ticker[['Open', 'Close']].max(axis=1)
    body_bottom = df_ticker[['Open', 'Close']].min(axis=1)
    pivot_high = (body_top == body_top.rolling(window, center=True).max())
    pivot_low = (body_bottom == body_bottom.rolling(window, center=True).min())
    df_ticker['pivot_high'] = pivot_high
    df_ticker['pivot_low'] = pivot_low
    resistance = df_ticker.loc[df_ticker['pivot_high'], ['Open', 'Close']].max(axis=1).tolist()
    support = df_ticker.loc[df_ticker['pivot_low'], ['Open', 'Close']].min(axis=1).tolist()

    # Group close levels
    def group_levels(levels):
        levels = sorted(levels)
        grouped = []
        for lvl in levels:
            if not grouped or abs(lvl - grouped[-1]) / grouped[-1] > buffer:
                grouped.append(lvl)
        return grouped

    results[ticker] = {
        "support": group_levels(support),
        "resistance": group_levels(resistance)
    }


In [ ]:
df_wide.groupby('Ticker').resample('1d', on='Timestamp')['Close'].last().dropna()